In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import numpy as np
from great_tables import GT, md, html, vals, loc, style, exibble

In [ ]:
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json"

df = pd.read_json(url)
df.head()

In [ ]:
df.info()

In [ ]:
# Normalizing the data frame
customer_df = pd.json_normalize(df['customer'])
phone_df = pd.json_normalize(df['phone'])
internet_df = pd.json_normalize(df['internet'])
account_df = pd.json_normalize(df['account'])

# Add customerID to each normalized dataframe for merging
customer_df['customerID'] = df['customerID']
phone_df['customerID'] = df['customerID']
internet_df['customerID'] = df['customerID']
account_df['customerID'] = df['customerID']

# Merge the dataframes
df = df[['customerID', 'Churn']].merge(customer_df, on='customerID').merge(phone_df, on='customerID').merge(internet_df, on='customerID').merge(account_df, on='customerID')

df.head()


In [ ]:
df.columns = df.columns.str.lower().str.replace('.','_')
df = df.rename(columns={'customerid':'customer_id','seniorcitizen':'senior_citizen','phoneservice':'phone_service','multiplelines':'multiple_lines','internetservice':'internet_service',
          'onlinesecurity':'online_security','onlinebackup':'online_backup','deviceprotection':'device_protection','techsupport':'tech_support',
          'streamingtv':'streaming_tv','streamingmovies':'streaming_movies','paperlessbilling':'paper_less_billing','paymentmethod':'payment_method'})
df.info()


In [ ]:
df['daily_accounts'] = (df.charges_monthly/30.44).round(2)
df.head()

In [ ]:

for col in df.columns:
  print(f'Quantity of unique values on columns "{col}": {df[col].nunique()}')
  if df[col].nunique() <50:
    print(df[col].unique())
  print(100*'-')

In [ ]:
df['charges_total'] = pd.to_numeric(df['charges_total'], errors='coerce')
df["charges_total"] = df["charges_total"].fillna(0)

for col in df.columns:
  print(f'Quantity of null values in column "{col}": {df[col].isnull().sum()}')
  print(50*'-')


In [ ]:
churn_filter = df[df['churn'].isin(['Yes','No'])].copy()
churn_filter['churn'] = churn_filter['churn'].replace({'No': 'Retained', 'Yes': 'Churned'})


Análise

In [ ]:
ax = sns.kdeplot(data=churn_filter, x="tenure", hue="churn", multiple="stack",
                 palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})


sns.move_legend(ax, 'upper center', bbox_to_anchor=(0.5, 1.2), ncol=3, title=None, frameon=False)
for text in ax.legend_.texts:
    text.set_fontsize(12)

ax.set_title('Tenure distribution', fontsize=18, pad=12)
ax.set_xlabel('Tenure')
ax.spines[['top', 'right']].set_visible(False)

plt.show()

In [ ]:
ax = sns.kdeplot(data=churn_filter, x="tenure", hue="churn", multiple="stack",
                 palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})


sns.move_legend(ax, 'upper center', bbox_to_anchor=(0.5, 1.2), ncol=3, title=None, frameon=False)
for text in ax.legend_.texts:
    text.set_fontsize(12)

ax.set_title('Tenure distribution', fontsize=18, pad=12)
ax.set_xlabel('Tenure')
ax.spines[['top', 'right']].set_visible(False)

plt.show()

In [ ]:
counts, bins = np.histogram(df[df['churn']=='Yes']['tenure'], bins=20)
mean_churn_count = counts.mean().round(0)

fig, ax1 = plt.subplots(figsize=(14,6))

sns.histplot(ax=ax1, data=churn_filter, x="tenure", hue="churn", bins=20, multiple='dodge', shrink=1, palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})
line = ax1.axhline(y=mean_churn_count, color='red', linestyle='--', label=f'Mean Churn : {mean_churn_count:.2f}')

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels + [line.get_label()], loc='upper center', bbox_to_anchor=(0.5, 1), ncol=3, title=None, frameon=False, fontsize=12)

ax1.spines[['top', 'right']].set_visible(False)
ax1.set_title('Histogram of Customer Churn by Tenure', fontsize=18)
ax1.set_xlabel('Tenure (Months)')
ax1.set_ylabel('Number of Customers')

plt.show()


In [ ]:
churn_rate_by_tenure = (churn_filter.groupby('tenure')['churn'].value_counts(normalize=True)*100).unstack()

plt.figure(figsize=(18, 8))
sns.lineplot(x=churn_rate_by_tenure.index, y=churn_rate_by_tenure['Churned'], marker='o', color='#dd8452')

ax = plt.gca()
formatter = mtick.FuncFormatter(lambda y, _: f'{y:.2f}'.replace('.', ',') + '%')
ax.yaxis.set_major_formatter(formatter)

plt.title('Churn Rate by time of tenure', fontsize=16)
plt.xlabel('Tenure (Months)', fontsize=12)
plt.ylabel('Churn Rate', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


In [ ]:
churn_rate = round((churn_filter.churn.value_counts(normalize= True)*100),2).to_frame()
colors = ['#4c72b0', '#dd8452']

fig, ax = plt.subplots(figsize=(9,6))

wedges, texts, autotexts = ax.pie(
    churn_rate['proportion'], labels=churn_rate.index, autopct='%1.1f%%', startangle=90,
    colors=colors
)

plt.setp(autotexts, size=14, weight='bold', color='w')
plt.setp(texts, size=14)

ax.set_title('Overal customer churn', fontsize=18, pad=12)
ax.yaxis.set_visible(False)
ax.spines[['top', 'left', 'right']].set_visible(False)

plt.show()

In [ ]:
sex_churn_rate = round(churn_filter.groupby('gender')[['churn']].value_counts(normalize=True)*100,2).reset_index()
pivoted_table = sex_churn_rate.pivot(index="gender",columns="churn",values="proportion").reset_index()

(
    GT(pivoted_table)
    .tab_header(
        title="Churn Rate by Gender",
    )
    .tab_spanner(
        label="Churn Status",
        columns=["Retained", "Churned"]
    )
    .fmt_number(
        columns=["Retained", "Churned"],
        decimals=2,
        pattern="{x} %"
    )
    .data_color(
        columns="Churned",
        palette=["#fdebea", "#e63946"]
    )
    .cols_label(
        gender="Gender",

    )
    .tab_options(
        heading_title_font_size="30px",
        column_labels_font_weight="bold",
        column_labels_font_size="20px",
        table_font_size='20px',
        table_border_top_style="hidden",
        table_border_bottom_style="hidden"
    )

)

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))

sns.barplot(data=sex_churn_rate, ax=ax, x='gender', y='proportion', hue='churn',
            palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})

for cont in ax.containers:
  ax.bar_label(cont, fmt='%.2f%%', label_type='edge', padding=5, fontsize=12, color='#333333')

sns.move_legend(ax, 'upper center', bbox_to_anchor=(0.5, 1.2), ncol=3, title=None, frameon=False)
for text in ax.legend_.texts:
    text.set_fontsize(12)

ax.set_title('Churn Rate by Gender', fontsize=18, pad=12)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax.set_ylim(0, max(sex_churn_rate.proportion)*1.1)
ax.yaxis.set_visible(False)
ax.set_xlabel('')
ax.spines[['top', 'left', 'right']].set_visible(False)

plt.show()

In [ ]:
churn_sitizien = (df.assign(senior_status = churn_filter.senior_citizen.map({0:'Under 65 years old', 1:'From 65 years old'}))
                .groupby('senior_status')[['churn']].value_counts(normalize=True).mul(100).round(2)).to_frame().reset_index()

s_pivoted_table = churn_sitizien.pivot(index="senior_status",columns="churn",values="proportion").reset_index()

(
    GT(s_pivoted_table)
    .tab_header(
        title="Churn Rate Senior Status",
    )
    .tab_spanner(
        label="Churn Status",
        columns=["No", "Yes"]
    )
    .fmt_number(
        columns=["Retained", "Churned"],
        decimals=2,
        pattern="{x} %"
    )
    .data_color(
        columns="Yes",
        palette=["#fdebea", "#e63946"]
    )
    .cols_label(
        senior_status="Senior Status",
        No = 'Retained',
        Yes= 'Churned'
    )
    .tab_options(
        heading_title_font_size="30px",
        column_labels_font_weight="bold",
        column_labels_font_size="20px",
        table_font_size='20px',
        table_border_top_style="hidden",
        table_border_bottom_style="hidden"
    )

)



In [ ]:
churn_sitizien['churn'] = churn_sitizien['churn'].replace({'No': 'Retained', 'Yes': 'Churned'})
fig, ax = plt.subplots(figsize=(9,6))

ax = sns.barplot(data=churn_sitizien, x='senior_status', y='proportion', hue='churn',
                 palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})

for cont in ax.containers:
  ax.bar_label(cont, fmt='%.2f%%', label_type='edge', padding=5, fontsize=12, color='#333333')

sns.move_legend(ax, 'upper center', bbox_to_anchor=(0.5, 1.2), ncol=3, title=None, frameon=False)
for text in ax.legend_.texts:
    text.set_fontsize(12)

ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax.set_title('Churn Rate by Senior level', fontsize=18, pad=12)
ax.set_xlabel('')
ax.set_ylim(0, max(sex_churn_rate.proportion)*1.1)
ax.yaxis.set_visible(False)
ax.spines[['top', 'left', 'right']].set_visible(False)

plt.show()


In [ ]:
contract_churn_rate = round(churn_filter.groupby('contract')[['churn']].value_counts(normalize=True)*100,2).to_frame().reset_index()
c_pivoted_table = contract_churn_rate.pivot(index="contract",columns="churn",values="proportion").reset_index()

(
    GT(c_pivoted_table)
    .tab_header(
        title="Churn Rate by Contract Type",
    )
    .tab_spanner(
        label="Churn Status",
        columns=["Retained", "Churned"]
    )
    .fmt_number(
        columns=["Retained", "Churned"],
        decimals=2,
        pattern="{x} %"
    )
    .data_color(
        columns="Churned",
        palette=["#fdebea", "#e63946"]
    )
    .cols_label(
        contract="Contract",

    )
    .tab_options(
        heading_title_font_size="30px",
        column_labels_font_weight="bold",
        column_labels_font_size="20px",
        table_font_size='20px',
        table_border_top_style="hidden",
        table_border_bottom_style="hidden"
    )

)

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))
ax = sns.barplot(data=contract_churn_rate, x='contract', y='proportion', hue='churn',
                 palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})

for cont in ax.containers:
  ax.bar_label(cont, fmt='%.2f%%', label_type='edge', padding=5, fontsize=12, color='#333333')

sns.move_legend(ax, 'upper center', bbox_to_anchor=(0.5, 1.2), ncol=3, title=None, frameon=False)
for text in ax.legend_.texts:
    text.set_fontsize(12)

ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))

ax.set_title('Churn Rate by contract type', fontsize=18, pad=12)
ax.set_xlabel('')
ax.yaxis.set_visible(False)
ax.spines[['top', 'left', 'right']].set_visible(False)

plt.show()


In [ ]:
payment_churn_rate = round(churn_filter.groupby('payment_method')[['churn']].value_counts(normalize=True)*100,2).to_frame().reset_index()
p_pivoted_table = payment_churn_rate.pivot(index="payment_method", columns="churn", values="proportion").reset_index()

(
    GT(p_pivoted_table)
    .tab_header(
        title="Churn Rate by Payment Method",
    )
    .tab_spanner(
        label="Churn Status",
        columns=["Retained", "Churned"]
    )
    .fmt_number(
        columns=["Retained", "Churned"],
        decimals=2,
        pattern="{x} %"
    )
    .data_color(
        columns="Churned",
        palette=["#fdebea", "#e63946"]
    )
    .cols_label(
        payment_method="Payment Method",

    )
    .tab_options(
        heading_title_font_size="30px",
        column_labels_font_weight="bold",
        column_labels_font_size="20px",
        table_font_size='20px',
        table_border_top_style="hidden",
        table_border_bottom_style="hidden"
    )

)

In [ ]:

fig, ax = plt.subplots(figsize=(12,6))
ax = sns.barplot(data=payment_churn_rate, x='payment_method', y='proportion', hue='churn',
                 palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})

for cont in ax.containers:
  ax.bar_label(cont, fmt='%.2f%%', label_type='edge', padding=5, fontsize=12, color='#333333')

sns.move_legend(ax, 'upper center', bbox_to_anchor=(0.5, 1.2), ncol=3, title=None, frameon=False)
for text in ax.legend_.texts:
    text.set_fontsize(12)

ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))

ax.set_title('Churn Rate by Payment Type', fontsize=18, pad=12)
ax.set_xlabel('')
ax.yaxis.set_visible(False)
ax.spines[['top', 'left', 'right']].set_visible(False)

plt.show()

In [ ]:
counts, bins = np.histogram(df[df['churn']=='Yes']['charges_total'], bins=20)
mean_churn_count = counts.mean().round(0)

fig, ax1 = plt.subplots(figsize=(14,6))

sns.histplot(ax=ax1, data=churn_filter, x="charges_total", hue="churn", bins=20, multiple='dodge', shrink=1, palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})

handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels + [line.get_label()], loc='upper center', bbox_to_anchor=(0.5, 1), ncol=3, title=None, frameon=False, fontsize=12)


ax1.spines[['top', 'right']].set_visible(False)
ax1.set_title('Histogram of Customer Churn by Total Charges', fontsize=18)
ax1.set_xlabel('Total Charges ($)')
ax1.set_ylabel('Number of Customers')

plt.show()


In [ ]:
internet_churn_rate = round(churn_filter.groupby('internet_service')[['churn']].value_counts(normalize=True)*100,2).to_frame().reset_index()
internet_churn_rate.internet_service = internet_churn_rate.internet_service.replace({'No':'No internet Service'})
i_pivoted_table = internet_churn_rate.pivot(index="internet_service", columns="churn", values="proportion").reset_index()

(
    GT(i_pivoted_table)
    .tab_header(
        title="Churn Rate by Payment Method",
    )
    .tab_spanner(
        label="Churn Status",
        columns=["Retained", "Churned"]
    )
    .fmt_number(
        columns=["Retained", "Churned"],
        decimals=2,
        pattern="{x} %"
    )
    .data_color(
        columns="Churned",
        palette=["#fdebea", "#e63946"]
    )
    .cols_label(
        internet_service="Internet Service",

    )
    .tab_options(
        heading_title_font_size="30px",
        column_labels_font_weight="bold",
        column_labels_font_size="20px",
        table_font_size='20px',
        table_border_top_style="hidden",
        table_border_bottom_style="hidden"
    )

)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax = sns.barplot(data=internet_churn_rate, x='internet_service', y='proportion', hue='churn',
                 palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})

for cont in ax.containers:
  ax.bar_label(cont, fmt='%.2f%%', label_type='edge', padding=5, fontsize=12, color='#333333')

sns.move_legend(ax, 'upper center', bbox_to_anchor=(0.5, 1.2), ncol=3, title=None, frameon=False)
for text in ax.legend_.texts:
    text.set_fontsize(12)

ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))

ax.set_title('Churn Rate by Internet Service', fontsize=18, pad=12)
ax.set_xlabel('')
ax.yaxis.set_visible(False)
ax.spines[['top', 'left', 'right']].set_visible(False)

plt.show()

In [ ]:
dependents_churn_rate = round(churn_filter.groupby('dependents')[['churn']].value_counts(normalize=True)*100,2).to_frame().reset_index()
dependents_churn_rate.dependents = dependents_churn_rate.dependents.replace({'No':'Does not have dependents', 'Yes':'Has dependents'})
d_pivoted_table = dependents_churn_rate.pivot(index="dependents", columns="churn", values="proportion").reset_index()

(
    GT(d_pivoted_table)
    .tab_header(
        title="Churn Rate by Dependents",
    )
    .tab_spanner(
        label="Churn Status",
        columns=["Retained", "Churned"]
    )
    .fmt_number(
        columns=["Retained", "Churned"],
        decimals=2,
        pattern="{x} %"
    )
    .data_color(
        columns="Churned",
        palette=["#fdebea", "#e63946"]
    )
    .cols_label(
        dependents="Dependents"

    )
    .tab_options(
        heading_title_font_size="30px",
        column_labels_font_weight="bold",
        column_labels_font_size="20px",
        table_font_size='20px',
        table_border_top_style="hidden",
        table_border_bottom_style="hidden"
    )

)


In [ ]:
pertner_churn_rate = round(churn_filter.groupby('partner')[['churn']].value_counts(normalize=True)*100,2).to_frame().reset_index()
pertner_churn_rate.partner = pertner_churn_rate.partner.replace({'No':'Does not have partner', 'Yes':'Has partner'})
p_pivoted_table = pertner_churn_rate.pivot(index="partner", columns="churn", values="proportion").reset_index()

(
    GT(p_pivoted_table)
    .tab_header(
        title="Churn Rate by Dependents",
    )
    .tab_spanner(
        label="Churn Status",
        columns=["Retained", "Churned"]
    )
    .fmt_number(
        columns=["Retained", "Churned"],
        decimals=2,
        pattern="{x} %"
    )
    .data_color(
        columns="Churned",
        palette=["#fdebea", "#e63946"]
    )
    .cols_label(
        partner="Partner"

    )
    .tab_options(
        heading_title_font_size="30px",
        column_labels_font_weight="bold",
        column_labels_font_size="20px",
        table_font_size='20px',
        table_border_top_style="hidden",
        table_border_bottom_style="hidden"
    )

)


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))

sns.barplot(data=dependents_churn_rate, x='dependents', y='proportion', hue='churn', ax=ax1,
                 palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})

for cont in ax1.containers:
  ax1.bar_label(cont, fmt='%.2f%%', label_type='edge', padding=5, fontsize=12, color='#333333')

sns.move_legend(ax1, 'upper center', bbox_to_anchor=(0.5, 1.2), ncol=2, title=None, frameon=False)
for text in ax1.legend_.texts:
    text.set_fontsize(12)

ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))

ax1.set_title('Churn Rate by Dependent Status', fontsize=18, pad=12)
ax1.set_xlabel('')
ax1.set_xticks(ax1.get_xticks())
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
ax1.yaxis.set_visible(False)
ax1.spines[['top', 'left', 'right']].set_visible(False)

sns.barplot(data=pertner_churn_rate, x='partner', y='proportion', hue='churn', ax=ax2,
                 palette={'Retained': '#4c72b0', 'Churned': '#dd8452'})

for cont in ax2.containers:
  ax2.bar_label(cont, fmt='%.2f%%', label_type='edge', padding=5, fontsize=12, color='#333333')

sns.move_legend(ax2, 'upper center', bbox_to_anchor=(0.5, 1.2), ncol=2, title=None, frameon=False)
for text in ax2.legend_.texts:
    text.set_fontsize(12)

ax2.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))

ax2.set_title('Churn Rate by Partner Status', fontsize=18, pad=12)
ax2.set_xlabel('')
# Explicitly set ticks before setting labels to avoid UserWarning
ax2.set_xticks(ax2.get_xticks())
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
ax2.yaxis.set_visible(False)
ax2.spines[['top', 'left', 'right']].set_visible(False)

plt.show()


#💡 Insights principais (do relatório)

Contrato Month-to-month aparece com churn mais alto que contratos de 1 e 2 anos.

Electronic check tende a ter churn acima da média comparado a outros métodos de pagamento.

Clientes com tenure baixo (≤ 6 meses) concentram mais churn.

Diferenças por tipo de internet (ex.: Fiber optic) merecem atenção em estratégia de retenção.

#Próximos passos sugeridos para o time de DS

Modelagem: logistic regression de baseline, depois tree-based (RF, XGBoost, LightGBM).

Feature engineering: interações (contrato × pagamento × tenure), normalização de numéricas, target encoding de categóricas.

Validação: estratificar por churn, métricas AUC/PR-AUC, calibration de probabilidades.

Ação: campanhas de retenção focadas em Month-to-month + Electronic check + Tenure baixo, com ofertas de upgrade para contratos mais longos e melhoria da experiência nos serviços de maior churn.